# Assignment 1

General Structure

    Parameters
    ----------
    A : str
        Element at the cube corners (A-site).
    B : str
        Element at the body center (B-site).
    c_to_a_ratio : float
        Ratio of c/a for the tetragonal phase.
    save_cifs : bool, optional
        If True, saves both structures as CIF files.
    show_widgets : bool, optional
        If True, displays 3D interactive structures using StructureWidget.

    Returns
    -------
    tuple[Structure, Structure]
        (cubic_perovskite, tetragonal_perovskite)



In [26]:
from pymatgen.core import Lattice, Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatviz.widgets.structure import StructureWidget

#c_to_a_ratio=1.3
def generate_perovskites(A: str, B: str, c_to_a_ratio: float, save_cifs: bool = True, show_widgets: bool = True):
    # --- 1. Define lattice constants ---
    a = 4.0
    c = a * c_to_a_ratio

    # --- 2. Atomic positions ---
    frac_coords = [
        (0, 0, 0),               # A-site
        (0.5, 0.5, 0.5),         # B-site
        (0.5, 0.5, 0),           # O1
        (0.5, 0, 0.5),           # O2
        (0, 0.5, 0.5)            # O3
    ]
    species = [A, B, "O", "O", "O"]

    # --- 3. Create structures ---
    cubic = Structure(Lattice.cubic(a), species, frac_coords)
    tetragonal = Structure(Lattice.tetragonal(a, c), species, frac_coords)

    # --- 4. Space group analysis ---
    cubic_sg = SpacegroupAnalyzer(cubic).get_space_group_symbol()
    tetragonal_sg = SpacegroupAnalyzer(tetragonal).get_space_group_symbol()

    print(f"Cubic space group: {cubic_sg}")
    print(f"Tetragonal space group: {tetragonal_sg}")

    # --- 5. Save CIFs ---
    if save_cifs:
        cubic.to(fmt="cif", filename=f"{A}{B}O3_cubic.cif")
        tetragonal.to(fmt="cif", filename=f"{A}{B}O3_tetragonal.cif")
        print("CIF files saved = YES")

    return cubic, tetragonal


# EDITABLE PARAMETERS
# Able to change elements, c/a ratio, and whether to save .cif files
cperov, tperov = generate_perovskites("Ba", "Ti", 1.3, save_cifs=True, show_widgets=True)


Cubic space group: Pm-3m
Tetragonal space group: P4/mmm
CIF files saved = YES


In [29]:
from IPython.display import display
from pymatviz.widgets.structure import StructureWidget

print("🟩 Cubic Perovskite:")
display(StructureWidget(cperov))

print("⬛ Tetragonal Perovskite:")
display(StructureWidget(tperov))

🟩 Cubic Perovskite:


StructureWidget(structure={'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattic…

⬛ Tetragonal Perovskite:


StructureWidget(structure={'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattic…

#### **It appears as though the .cif files saved correctly.** 

#### The only differece between them is the **cell length c**, which should be the correct difference between the cubic and tetregonal structures.

#### Also, I wanted to print the cell structure using structure widget, for some reason only the **tetragonal** image appears but **not** the cubic. I do not know why at all

# Assignment 2

In [8]:
from mp_api.client import MPRester
from emmet.core.symmetry import CrystalSystem
from emmet.core.mpid import MPID
import matplotlib.pyplot as plt
from pymatviz.widgets.structure import StructureWidget
import pandas as pd

ImportError: cannot import name 'Sentinel' from 'typing_extensions' (/Users/coenradttaylor/opt/anaconda3/envs/chme6320/lib/python3.11/site-packages/typing_extensions.py)

In [9]:
from mp_api.client import MPRester
API_KEY = "5b2toNu5RX6WD7M8QREXXnGVb0C0WpEv"
with MPRester(API_KEY) as mpr:
    docs = mpr.materials.summary.search(
        is_metal=False,
        theoretical=False,
        fields=["material_id", "band_gap", "symmetry", "is_gap_direct"]
    )
    data = docs  # list of summary documents


ImportError: cannot import name 'Sentinel' from 'typing_extensions' (/Users/coenradttaylor/opt/anaconda3/envs/chme6320/lib/python3.11/site-packages/typing_extensions.py)